In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

In [ ]:
train = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train" 
val ="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val"
test = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Test"

gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2,)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (220, 220))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(220, 220))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(220, 220))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights = 'imagenet', input_shape = (220, 220,3), include_top = False)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                           

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,restore_best_weights=True)

history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 25, callbacks = callback)

Epoch 1/25
40/40 [==============================] - 599s 15s/step - loss: 0.4283 - accuracy: 0.8039 - val_loss: 0.4287 - val_accuracy: 0.8006
Epoch 2/25
40/40 [==============================] - 296s 7s/step - loss: 0.3118 - accuracy: 0.8594 - val_loss: 0.4451 - val_accuracy: 0.7784
Epoch 3/25
40/40 [==============================] - 235s 6s/step - loss: 0.2689 - accuracy: 0.8844 - val_loss: 0.2524 - val_accuracy: 0.8753
Epoch 4/25
40/40 [==============================] - 223s 6s/step - loss: 0.2305 - accuracy: 0.8945 - val_loss: 0.2336 - val_accuracy: 0.8934
Epoch 5/25
40/40 [==============================] - 208s 5s/step - loss: 0.2106 - accuracy: 0.9090 - val_loss: 0.1875 - val_accuracy: 0.9169
Epoch 6/25
40/40 [==============================] - 217s 5s/step - loss: 0.2228 - accuracy: 0.9013 - val_loss: 0.1235 - val_accuracy: 0.9515
Epoch 7/25
40/40 [==============================] - 204s 5s/step - loss: 0.2167 - accuracy: 0.9137 - val_loss: 0.2570 - val_accuracy: 0.8726
Epoch 8/25
4

In [ ]:
model.save("MobileNet_v2_3.h5")

In [ ]:
np.save("MobileNet_history.npy",history.history)

In [ ]:
y_pred = model.predict(test_gen)

In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])



In [ ]:
y_actual = []

for i in range(len(y_pred)):
  if i <361 :
    y_actual.append(0)
  else:
    y_actual.append(1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,y_pred_list)

In [ ]:
np.save("Mobilenet-history.npy",history.history)
model.save("Mobilenet.h5")

In [ ]:
from google.colab import files 
files.download("Mobilenet.h5")

files.download("Mobilenet-history.npy")